# Training the data

In [115]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# load the csv file with user input and intent
data = pd.read_csv('input_intent.csv')

# split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

# vectorize the user input using CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data['user_Input'])
X_test = vectorizer.transform(test_data['user_Input'])

# train a logistic regression model
y_train = train_data['Intent']
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

# Install and import

In [86]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 593.3 kB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [103]:
import spacy
nlp = spacy.load("en_core_web_sm")

# Define intent identifiers
INTENT_FLIGHT_BOOKING = 1
INTENT_HOTEL_BOOKING = 2
INTENT_GREETING = 3
INTENT_CHANGE_DATE = 4
INTENT_CHANGE_DESTINATION = 5
INTENT_END_BOOKING = 8
INTENT_CANCEL_FLIGHT = 7

# Define state identifiers
UNIDENTIFIED = -1
STATE_INITIAL = 0
STATE_FLIGHT_RESERVATION = 1
STATE_FLIGHT_CHANGE = 2
STATE_FLIGHT_CANCELLATION = 3
STATE_MISSING_DATE = 4
STATE_MISSING_DATE_DESTINATION = 6
STATE_MISSING_DESTINATION = 5
STATE_MISSING_DATE_CHANGE = 7
STATE_MISSING_DATE_DESTINATION_CHANGE = 8
STATE_MISSING_DESTINATION_CHANGE = 9
STATE_MISSING_DATE_CANCEL = 10
STATE_MISSING_DATE_DESTINATION_CANCEL = 11
STATE_MISSING_DESTINATION_CANCEL = 12
STATE_CONFIRMATION = 13


# Process the input from the user

In [104]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

def process_chat_input1(user_input, current_state):
    # initialize variables to be returned
    output = ""
    intent = ""
    next_state = ""
    entities = []
    polarity = ""

    # perform sentiment analysis
    sia = SentimentIntensityAnalyzer()
    scores = sia.polarity_scores(user_input)
    polarity_score = scores['compound']
    if polarity_score > 0:
        polarity = "positive"
    elif polarity_score < 0:
        polarity = "negative"
    else:
        polarity = "neutral"

    # perform NER
    doc = nlp(user_input)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            entities.append(('GPE', ent.text))
        elif ent.label_ == 'DATE':
            entities.append(('DATE', ent.text))

    # find the intent
    input_vec = vectorizer.transform([user_input])
    intent = model.predict(input_vec)[0]

    # find the next state
    if intent == 'Flight_Booking': #state 1
        if 'GPE' not in [e[0] for e in entities] and 'DATE' not in [e[0] for e in entities]: #state 6
            output = "Where and when do you want to travel?"
            next_state = 13
        elif 'GPE' not in [e[0] for e in entities]: #state 5
            output = "Where do you want to travel?"
            next_state = 13
        elif 'DATE' not in [e[0] for e in entities]: #state 4
            output = "When do you want to travel?"
            next_state = 13
        else:
            output = "Your flight has been booked. Thank you for choosing us!"
            next_state = 0 #to beginning

    elif intent == 'Hotel_Booking':
        if 'GPE' not in [e[0] for e in entities] and 'DATE' not in [e[0] for e in entities]: #state 6
            output = "Where and when do you want to book your hotel?"
            next_state = 13
        elif 'GPE' not in [e[0] for e in entities]: #state 5
            output = "Where do you want to book your hotel?"
            next_state = 13
        elif 'DATE' not in [e[0] for e in entities]: #state 4
            output = "When do you want to book your hotel?"
            next_state = 13
        else:
            output = "Your hotel has been booked. Thank you for choosing us!"
            next_state = 0

    elif intent == 'Change_Flight': #state 2
        if 'GPE' not in [e[0] for e in entities] and 'DATE' not in [e[0] for e in entities]: #state 8
            output = "Where and when do you want to change your flight?"
            next_state = 13
        elif 'GPE' not in [e[0] for e in entities]: #state 9
            output = "Where do you want to change your flight to?"
            next_state = 13
        elif 'DATE' not in [e[0] for e in entities]: #state 7
            output = "When do you want to change your flight?"
            next_state = 13
        else:
            output = "Your flight has been changed. Thank you for choosing us!"
            next_state = 0

    elif intent == 'Change_Destination': #state 2
        if 'GPE' not in [e[0] for e in entities]: #state 9
            output = "Where do you want to change your destination to?"
            next_state = 13
        else:
            output = "Your destination has been changed. Thank you for choosing us!"
            next_state = 0
    elif intent == 'Change_Date': #state 2
        if 'DATE' not in [e[0] for e in entities]: #state 7
            output = "When do you want to change your destination to?"
            next_state = 13
        else:
            output = "Your destination has been changed. Thank you for choosing us!"
            next_state = 0
    elif intent == 'cancel_flight': #state 3
        if 'GPE' not in [e[0] for e in entities] and 'DATE' not in [e[0] for e in entities]: #state 11
            output = "Where and when do you want to cancel your flight?"
            next_state = 13
        elif 'DATE' not in [e[0] for e in entities]: #state 12
            output = "When do you want to cancel your booking?"
            next_state = 13
        elif 'GPE' not in [e[0] for e in entities]: #state 10
            output = "Where do you want to cancel your destination to?"
            next_state = 13
        else:
            output = "Your booking has been cancelled. Thank you for choosing us!"
            next_state = 0
    elif intent == 'end_booking':
        output = "Thank you for using our service."
        next_state = 0 #to beginning
    elif current_state == 0:
        output = "Can i help you with something else?"
        next_state = 0
    elif current_state == 13: #confirmation
        output = "Your request confirm! Enter exit if you finish or write something else that you want:)"
        next_state = 0
    else:
        output = "Sorry, I don't understand what you mean by that."
        next_state = -1

    return output, intent, next_state, entities, polarity


# Online mode

In [117]:
def run_online():
    flag = True
    output=''
    name=''
    continue_d = True
    current_state = 0
    while continue_d:
        if flag:
            output="Hello, what is your name?"
            print("Chatbot: " +output)
            name=input(output)
            print("User: "+name)
            output= f'Hello {name}, How can i help you?'
            print("Chatbot: "+output)
            flag = False
        if current_state == -1:
            continue_d=False
            print("Chatbot: GoodBye!")
        else:
            user_input=input(output)
            user_input= user_input.lower()
            if user_input=="exit":
                continue_d= False
                print("Chatbot: GoodBye!")
            else:
                print("User: "+user_input)
                output, intent, next_state, entities, polarity = process_chat_input1(user_input, current_state)
                print("Chatbot: " +output)
                current_state = next_state

# run_online()

# Offline mode

In [128]:
import pandas as pd

def run_offline_chatbot(input_file_path, output_file_path):
    current_state = 0
    df = pd.read_csv(input_file_path)

    responses = []
    for index, row in df.iterrows():
        output, intent, next_state, entities, polarity = process_chat_input1(row['input'], current_state)
        current_state = next_state
        response = {
            "output": output,
            "intent": intent,
            "next_state":next_state,
            "entities": entities,
            "ploralty": polarity
        }
        responses.append(response)

    # Write the responses to a CSV file
    response_df = pd.DataFrame(responses)
    response_df.to_csv(output_file_path, index=False)

input_file_path="input1.csv"
output_file_path="output1.csv"
run_offline_chatbot(input_file_path, output_file_path)